# 获取数据集

In [86]:
import zipfile
r = requests.get('https://github.com/cystanford/text_classification/archive/master.zip',stream = True)
print(r.status_code)
f = io.BytesIO(r.content)
names = zipfile.ZipFile(f).namelist()
print(names[0:5])

200


['text_classification-master/',
 'text_classification-master/README.md',
 'text_classification-master/text classification/',
 'text_classification-master/text classification/stop/',
 'text_classification-master/text classification/stop/stopword.txt']

In [418]:
def read_txt(filename, encoding='utf-8'):
    with zipfile.ZipFile(f) as myzip:
        with myzip.open(filename, 'r') as myfile:
            lines = myfile.readlines()

    # print(filename)
    stop_words = [line.strip().decode(encoding, 'ignore')
                  for line in lines]  #'ignore'
    return stop_words


stop_words_2 = read_txt(
    'text_classification-master/text classification/stop/stopword.txt',
    encoding='utf-8-sig')
print(' '.join([x for x in stop_words_2][0:10]))

, ? 、 。 “ ” 《 》 ！ ，


In [256]:
from itertools import chain

def read_folder(reg_str):
    folder_names = list(filter(re.compile(reg_str).match, names))
    print(folder_names)
    data = []
    for ifolder_name in folder_names:
        data_files = list(filter(re.compile(ifolder_name + ".+$").match, names))
        # print(data_files[0])
        idata = [read_txt(idata_file, encoding='gbk') for idata_file in data_files]
        data.append(list(chain(*idata)))
        
    return data

In [192]:
train = read_folder(".*train/.+/$")
test = read_folder(".*test/.+/$")
print(train[0][:5])
print('_'*40)
print(test[0][:5])

['球场禁用招数“少先队员之惩戒”冠', '可以直接到编辑部买，地址，北京体育馆路8号，中国体育报业总社院内，后楼51700：羽毛球杂志木有支付宝，木有财付通，在网上订购不支持货到付款么？那么北京哪个实体店有卖12月的《羽毛球》杂志，或者说，能去你们编辑社买不？地址？转发(3)评论(4)12月10日09:17来自新浪微博', '直播贴阿内尔卡正式加盟申花，最现场连线申花新闻官带来第一手消息，上海男篮2分惜败惨遭4连败，北京延续不败战绩欧洲足坛激战正酣，米兰被弱旅逼平，拜仁重夺联赛榜首风云变幻，霍华德要求离队，湖人退出三方交易更多内容请听11:00的《990体育新闻》934:', '组图：金妍儿黑丝亮相热心公益 OL套装透成熟 http:url.cn/1bNQDP  (分享自 腾讯体育 )$LOTOzf$', '北京23分落后末节大反扑 惜败佛山终结13连胜  http:url.cn/2nGdrq  (分享自 腾讯体育 )$LOTOzf$']
________________________________________
['意大利的新人Edoardo Alescio上周末成为WPT冠军俱乐部的一员。他在WPT威尼斯站点的比赛获得了冠军和奖金17.5万欧元。$LOTOzf$', '李娜真棒 坚持到了最后 国歌在法网比赛中第一次响起 好激动哈$LOTOzf$', '北京时间3月4日，凯尔特人队消息，前锋“大宝贝”格伦-戴维斯将因为左膝髌骨肌腱炎缺席一周。“大宝贝”本赛季场均11.6分5.3篮板，他是在115-103击败太阳队的比赛中受伤的，但伤势并不严重。$LOTOzf$', '是山东队的吴轲，本赛季进步非常大，赛季开始前还入选了范斌执教的国奥队前往甘肃兰州和NBL明星队打了一场全明星赛。$LOTOzf$此微博已被原作者删除。', '有点危险，今天看斯洛文尼亚的记者会，有可能打类似朝鲜阵型（或龟缩战术），因为斯洛文尼亚战平就出线，且是历史上第一次世界杯小组出线，肯定会死守。会上的言语还间接挑衅了鲁尼，称其徒有虚名（心理战），鲁小胖脾气暴，是有目共睹，但愿别急于求成，在比赛中上当。打出配合，赢球不难。$LOTOzf$']


# 整理数据集

In [419]:
import jieba

# 分类label
categories = ['sports', 'female', 'literature', 'campus']

def get_labels(data):
    data_labels = [[icategory] * len(idata)
                   for idata, icategory in zip(data, categories)]
    return list(chain(*data_labels))

train_labels = get_labels(train)
test_labels = get_labels(test)
print(train_labels[:5])
print('_' * 40)
print(test_labels[:5])

# 分类features分词
def get_words(data):
    data_chain = list(chain(*data))
    # 分词 & 去掉非中文字符
    data_chain = [
        re.compile(u'[^\u4E00-\u9FFF]').sub(
            r' ', ' '.join(jieba.cut(x)))
        for x in data_chain
    ]
    print(data_chain[:5])
    print('_' * 40)
    return data_chain

train_chain = get_words(train)
test_chain = get_words(test)

['sports', 'sports', 'sports', 'sports', 'sports']
________________________________________
['sports', 'sports', 'sports', 'sports', 'sports']
['球场 禁用 招数   少先队员 之 惩戒   冠', '可以 直接 到 编辑部 买   地址   北京 体育馆 路   号   中国 体育 报业 总社 院内   后楼         羽毛球 杂志 木有 支付宝   木有 财付 通   在 网上 订购 不 支持 货到付款 么   那么 北京 哪个 实体店 有 卖    月 的   羽毛球   杂志   或者说   能 去 你们 编辑 社买 不   地址   转发       评论          月    日         来自 新浪 微博', '直播 贴 阿内尔卡 正式 加盟 申花   最 现场 连线 申花 新闻官 带来 第一手 消息   上海 男篮   分 惜败 惨遭   连败   北京 延续 不败 战绩 欧洲 足坛 激战 正酣   米兰 被 弱旅 逼平   拜仁 重夺 联赛 榜首 风云变幻   霍华德 要求 离队   湖人 退出 三方 交易 更 多 内容 请 听         的       体育新闻        ', '组图   金妍儿 黑丝 亮相 热心 公益      套装 透 成熟                                  分享 自   腾讯 体育               ', '北京    分 落后 末节 大 反扑   惜败 佛山 终结    连胜                                    分享 自   腾讯 体育               ']
________________________________________
['意大利 的 新人                   上周末 成为     冠军 俱乐部 的 一员   他 在     威尼斯 站点 的 比赛 获得 了 冠军 和 奖金      万 欧元             ', '李娜 真棒   坚持 到 了 最后   国歌 在 法网 比赛 中 第一次 响起   好 激动 哈     

# 提取特征

In [420]:
tfidf_vec_train = TfidfVectorizer(stop_words=stop_words_2, max_df=0.5, analyzer='word')
tfidf_matrix_train = tfidf_vec_train.fit_transform(train_chain)
print(tfidf_vec_train.get_feature_names()[:10])
train_features = tfidf_matrix_train.toarray()
print(train_features.shape)

train_vocabulary = tfidf_vec_train.vocabulary_
tfidf_vec_test = TfidfVectorizer(
    stop_words=stop_words_2, max_df=0.5, vocabulary=train_vocabulary)
tfidf_matrix_test = tfidf_vec_test.fit_transform(test_chain)
print(tfidf_vec_train.get_feature_names()[:10])
test_features = tfidf_matrix_test.toarray()
print(test_features.shape)

/Users/mymac/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['一一', '一丁', '一万', '一万元', '一上午', '一下', '一下下', '一下子', '一下脸', '一丝']
(3313, 21004)
['一一', '一丁', '一万', '一万元', '一上午', '一下', '一下下', '一下子', '一下脸', '一丝']
(200, 21004)


/Users/mymac/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# 多项式贝叶斯分类器

In [421]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB  
model = MultinomialNB(alpha=0.001).fit(train_features, train_labels)

prediction = model.predict(test_features)
print('The accuracy of the MultinomialNB is',
      metrics.accuracy_score(prediction, test_labels))
print(metrics.classification_report(test_labels, prediction))
print(metrics.confusion_matrix(test_labels, prediction))

The accuracy of the MultinomialNB is 0.92
              precision    recall  f1-score   support

      campus       0.92      0.69      0.79        16
      female       0.92      0.95      0.94        38
  literature       0.79      0.97      0.87        31
      sports       0.96      0.93      0.95       115

   micro avg       0.92      0.92      0.92       200
   macro avg       0.90      0.88      0.88       200
weighted avg       0.93      0.92      0.92       200

[[ 11   0   3   2]
 [  0  36   1   1]
 [  0   0  30   1]
 [  1   3   4 107]]
